In [14]:
import sys
import torch.utils.data
import torch
from glob import glob
from  torchvision import transforms,datasets
from PIL import Image
import numpy as np
import os
import torch.nn as nn
import torch.nn.functional as F
import argparse
import time
import cv2
from torch.autograd import Variable
from tensorboardX import SummaryWriter
parser = argparse.ArgumentParser('pix4d')

## general settings.
parser.add_argument('--use_gpu', type=int, default=1)
parser.add_argument('--batch', type=int, default=1) #batch size
parser.add_argument('--num_epoch', type=int, default=5000) #number of epochs
config, _ = parser.parse_known_args()

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3 ,16, 3, padding=1)
        self.conv2 = nn.Conv2d(16,32, 3, padding=1)
        self.conv3 = nn.Conv2d(32,16, 3, padding=1)
        self.conv4 = nn.Conv2d(16, 2, 5, padding=2)
        self.max = nn.MaxPool2d(2, stride=2)
        self.ups = nn.Upsample(scale_factor=2, mode='bilinear')
        self.relu  = nn.ReLU()
        self.softmax = nn.Softmax2d()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.max(x)
        x = self.relu(self.conv3(x))
        x = self.ups(x)
        x = self.softmax(self.conv4(x))
        return x

In [16]:
class Pix4dDataloader(torch.utils.data.Dataset):

    def __init__(self,data_dir):
        self.data_dir=data_dir
        self.image_names = glob(data_dir + "/img/*") 
        self.gt_names = glob(data_dir + "/gt/*")
        self.transforms = transforms.Compose([
            transforms.ToTensor(),
                                           ])

    def image_loader(self,image,gt,transform):
        
        image = transform(Image.open(image).convert('RGB').resize((256, 204), Image.BILINEAR))
        gt = transform(Image.open(gt).convert('LA').resize((256, 204), Image.NEAREST)).type(torch.LongTensor)
        c,h,w = gt.shape
        label = torch.zeros((2,h,w))
        for i in range(h):
            for j in range(w):
                label[gt[0][i][j]][i][j] = 1
        return image,label
    
    
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):

        images = '{}'.format(self.image_names[idx])
        gts = '{}'.format(self.gt_names[idx])
        images,gts = self.image_loader(images,gts,transform=self.transforms)
        return [images,gts]

In [18]:
"""
Training
"""

def train_model(net, train_loader,criterion,num_epoch,optimizer):
    print('<training start>')
    writer_loss = SummaryWriter('./log/loss')
    for epoch in range(num_epoch):
        since = time.time()
        running_train_loss = 0.0
        for i, (imgs,labels) in enumerate(train_loader):
            
            if config.use_gpu:
                imgs = Variable(imgs.cuda()).float()
                labels = Variable(labels.cuda()).float()

            else:
                imgs = Variable(imgs).float()
                labels = Variable(labels).float()

            optimizer.zero_grad()
            out = net(imgs)
            loss = criterion(out,labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()

        if epoch!= 0:
                epoch_time=time.time()-since
                print("Time:{:f} Epoch [{}/{}] Train: {:.4f}".format(
                                epoch_time,
                                epoch,
                                num_epoch,
                                float(running_train_loss)/train_num
                                ))
                
                writer_loss.add_scalar('Loss', float(running_train_loss)/train_num, epoch)

        if epoch%5==0:
                # save trained model
                torch.save(net.state_dict(), "trained_model/Net_" + str(epoch) + ".pkl")

        if epoch%5==0:    
                image = out.max(1)[1].detach()
                if config.use_gpu:
                    image = image.cpu().data.numpy().squeeze()
                else:
                    image = image.data.numpy().squeeze()
                cv2.imwrite('intermediate_results/'+str(epoch)+'.png',255*image)
    writer_loss.close()       
    print
    print('<Finished Training>')
    return 0



if __name__ == '__main__':
    path_dic={
        "train":'images',
        "val": 'images'
    }

    """
    Data loading
    """
    train = Pix4dDataloader(path_dic["train"])
    train_loader = torch.utils.data.DataLoader(train, batch_size=config.batch, num_workers=1)
    train_num = train.__len__()

    if config.use_gpu:
        net=Net().cuda()
    else:
        net=Net()
    
    lr = 0.0003
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.BCELoss()
    print(net)
    print("lr:",lr)
    start_time=time.time()
    train_model(net,train_loader,criterion, num_epoch=config.num_epoch,optimizer=optimizer)



Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(16, 2, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (max): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (ups): Upsample(scale_factor=2, mode=bilinear)
  (relu): ReLU()
  (softmax): Softmax2d()
)
lr: 0.0003
<training start>
Time:0.760767 Epoch [1/5000] Train: 0.6624
Time:0.778247 Epoch [2/5000] Train: 0.6537
Time:0.784219 Epoch [3/5000] Train: 0.6449
Time:0.789747 Epoch [4/5000] Train: 0.6358
Time:0.794853 Epoch [5/5000] Train: 0.6263
Time:0.770040 Epoch [6/5000] Train: 0.6165
Time:0.766705 Epoch [7/5000] Train: 0.6062
Time:0.792667 Epoch [8/5000] Train: 0.5955
Time:0.786554 Epoch [9/5000] Train: 0.5841
Time:0.777962 Epoch [10/5000] Train: 0.5719
Time:0.767972 Epoch [11/5000] Train: 0.5589
Time:0.78

Time:0.761390 Epoch [174/5000] Train: 0.1843
Time:0.762661 Epoch [175/5000] Train: 0.1835
Time:0.767956 Epoch [176/5000] Train: 0.1827
Time:0.757422 Epoch [177/5000] Train: 0.1819
Time:0.776072 Epoch [178/5000] Train: 0.1812
Time:0.772459 Epoch [179/5000] Train: 0.1804
Time:0.769181 Epoch [180/5000] Train: 0.1797
Time:0.771504 Epoch [181/5000] Train: 0.1789
Time:0.767322 Epoch [182/5000] Train: 0.1781
Time:0.763892 Epoch [183/5000] Train: 0.1774
Time:0.761176 Epoch [184/5000] Train: 0.1767
Time:0.762341 Epoch [185/5000] Train: 0.1759
Time:0.768182 Epoch [186/5000] Train: 0.1752
Time:0.767087 Epoch [187/5000] Train: 0.1745
Time:0.775048 Epoch [188/5000] Train: 0.1738
Time:0.763747 Epoch [189/5000] Train: 0.1731
Time:0.764891 Epoch [190/5000] Train: 0.1724
Time:0.776687 Epoch [191/5000] Train: 0.1717
Time:0.769300 Epoch [192/5000] Train: 0.1710
Time:0.782311 Epoch [193/5000] Train: 0.1703
Time:0.769015 Epoch [194/5000] Train: 0.1696
Time:0.763723 Epoch [195/5000] Train: 0.1689
Time:0.773

Time:0.897392 Epoch [357/5000] Train: 0.0861
Time:0.845571 Epoch [358/5000] Train: 0.0860
Time:0.849703 Epoch [359/5000] Train: 0.0859
Time:0.805031 Epoch [360/5000] Train: 0.0859
Time:0.822612 Epoch [361/5000] Train: 0.0858
Time:0.835145 Epoch [362/5000] Train: 0.0858
Time:0.859786 Epoch [363/5000] Train: 0.0857
Time:0.908866 Epoch [364/5000] Train: 0.0856
Time:0.826483 Epoch [365/5000] Train: 0.0855
Time:0.824297 Epoch [366/5000] Train: 0.0853
Time:0.819564 Epoch [367/5000] Train: 0.0852
Time:0.831879 Epoch [368/5000] Train: 0.0851
Time:0.868462 Epoch [369/5000] Train: 0.0850
Time:0.823453 Epoch [370/5000] Train: 0.0849
Time:0.822355 Epoch [371/5000] Train: 0.0849
Time:0.824831 Epoch [372/5000] Train: 0.0848
Time:0.834808 Epoch [373/5000] Train: 0.0848
Time:0.836575 Epoch [374/5000] Train: 0.0847
Time:0.840002 Epoch [375/5000] Train: 0.0846
Time:0.837856 Epoch [376/5000] Train: 0.0844
Time:0.837174 Epoch [377/5000] Train: 0.0843
Time:0.829606 Epoch [378/5000] Train: 0.0842
Time:0.814

Time:0.814073 Epoch [540/5000] Train: 0.0750
Time:0.819838 Epoch [541/5000] Train: 0.0750
Time:0.815598 Epoch [542/5000] Train: 0.0749
Time:0.804732 Epoch [543/5000] Train: 0.0749
Time:0.819161 Epoch [544/5000] Train: 0.0748
Time:0.804779 Epoch [545/5000] Train: 0.0747
Time:0.810472 Epoch [546/5000] Train: 0.0747
Time:0.813316 Epoch [547/5000] Train: 0.0747
Time:0.810508 Epoch [548/5000] Train: 0.0746
Time:0.812365 Epoch [549/5000] Train: 0.0746
Time:0.817719 Epoch [550/5000] Train: 0.0745
Time:0.810777 Epoch [551/5000] Train: 0.0744
Time:0.805543 Epoch [552/5000] Train: 0.0744
Time:0.812977 Epoch [553/5000] Train: 0.0743
Time:0.807089 Epoch [554/5000] Train: 0.0743
Time:0.807907 Epoch [555/5000] Train: 0.0742
Time:0.810914 Epoch [556/5000] Train: 0.0742
Time:0.814084 Epoch [557/5000] Train: 0.0741
Time:0.815875 Epoch [558/5000] Train: 0.0741
Time:0.812361 Epoch [559/5000] Train: 0.0740
Time:0.806463 Epoch [560/5000] Train: 0.0740
Time:0.807943 Epoch [561/5000] Train: 0.0739
Time:0.814

Time:0.810936 Epoch [723/5000] Train: 0.0658
Time:0.807860 Epoch [724/5000] Train: 0.0657
Time:0.804040 Epoch [725/5000] Train: 0.0657
Time:0.798826 Epoch [726/5000] Train: 0.0656
Time:0.791962 Epoch [727/5000] Train: 0.0656
Time:0.828231 Epoch [728/5000] Train: 0.0656
Time:0.797960 Epoch [729/5000] Train: 0.0655
Time:0.809636 Epoch [730/5000] Train: 0.0655
Time:0.802850 Epoch [731/5000] Train: 0.0654
Time:0.854690 Epoch [732/5000] Train: 0.0654
Time:0.808950 Epoch [733/5000] Train: 0.0653
Time:0.835977 Epoch [734/5000] Train: 0.0653
Time:0.799975 Epoch [735/5000] Train: 0.0652
Time:0.800180 Epoch [736/5000] Train: 0.0652
Time:0.809447 Epoch [737/5000] Train: 0.0651
Time:0.815155 Epoch [738/5000] Train: 0.0651
Time:0.805764 Epoch [739/5000] Train: 0.0650
Time:0.805060 Epoch [740/5000] Train: 0.0650
Time:0.792422 Epoch [741/5000] Train: 0.0649
Time:0.805119 Epoch [742/5000] Train: 0.0649
Time:0.799768 Epoch [743/5000] Train: 0.0649
Time:0.801426 Epoch [744/5000] Train: 0.0649
Time:0.798

Time:0.817155 Epoch [906/5000] Train: 0.0571
Time:0.844407 Epoch [907/5000] Train: 0.0568
Time:0.825114 Epoch [908/5000] Train: 0.0566
Time:0.810471 Epoch [909/5000] Train: 0.0565
Time:0.805802 Epoch [910/5000] Train: 0.0566
Time:0.811772 Epoch [911/5000] Train: 0.0568
Time:0.804168 Epoch [912/5000] Train: 0.0569
Time:0.797653 Epoch [913/5000] Train: 0.0569
Time:0.851486 Epoch [914/5000] Train: 0.0568
Time:0.848895 Epoch [915/5000] Train: 0.0566
Time:0.847445 Epoch [916/5000] Train: 0.0563
Time:0.838755 Epoch [917/5000] Train: 0.0562
Time:0.829808 Epoch [918/5000] Train: 0.0562
Time:0.860730 Epoch [919/5000] Train: 0.0562
Time:0.832025 Epoch [920/5000] Train: 0.0563
Time:0.824958 Epoch [921/5000] Train: 0.0563
Time:0.807845 Epoch [922/5000] Train: 0.0563
Time:0.794639 Epoch [923/5000] Train: 0.0561
Time:0.804921 Epoch [924/5000] Train: 0.0560
Time:0.805418 Epoch [925/5000] Train: 0.0558
Time:0.790545 Epoch [926/5000] Train: 0.0558
Time:0.796313 Epoch [927/5000] Train: 0.0558
Time:0.797

Time:0.817235 Epoch [1087/5000] Train: 0.0501
Time:0.806399 Epoch [1088/5000] Train: 0.0500
Time:0.813061 Epoch [1089/5000] Train: 0.0498
Time:0.805547 Epoch [1090/5000] Train: 0.0497
Time:0.812958 Epoch [1091/5000] Train: 0.0496
Time:0.835685 Epoch [1092/5000] Train: 0.0495
Time:0.815212 Epoch [1093/5000] Train: 0.0495
Time:0.817666 Epoch [1094/5000] Train: 0.0495
Time:0.799740 Epoch [1095/5000] Train: 0.0495
Time:0.823617 Epoch [1096/5000] Train: 0.0495
Time:0.813112 Epoch [1097/5000] Train: 0.0495
Time:0.821314 Epoch [1098/5000] Train: 0.0495
Time:0.808059 Epoch [1099/5000] Train: 0.0495
Time:0.800988 Epoch [1100/5000] Train: 0.0494
Time:0.844656 Epoch [1101/5000] Train: 0.0494
Time:0.816094 Epoch [1102/5000] Train: 0.0493
Time:0.824849 Epoch [1103/5000] Train: 0.0492
Time:0.801431 Epoch [1104/5000] Train: 0.0491
Time:0.799252 Epoch [1105/5000] Train: 0.0490
Time:0.828659 Epoch [1106/5000] Train: 0.0490
Time:0.799593 Epoch [1107/5000] Train: 0.0490
Time:0.809876 Epoch [1108/5000] Tr

Time:0.808152 Epoch [1266/5000] Train: 0.0442
Time:0.798276 Epoch [1267/5000] Train: 0.0442
Time:0.790777 Epoch [1268/5000] Train: 0.0442
Time:0.796472 Epoch [1269/5000] Train: 0.0442
Time:0.806072 Epoch [1270/5000] Train: 0.0441
Time:0.836372 Epoch [1271/5000] Train: 0.0441
Time:0.824440 Epoch [1272/5000] Train: 0.0441
Time:0.865439 Epoch [1273/5000] Train: 0.0440
Time:0.849972 Epoch [1274/5000] Train: 0.0440
Time:0.939812 Epoch [1275/5000] Train: 0.0440
Time:0.823011 Epoch [1276/5000] Train: 0.0440
Time:0.891859 Epoch [1277/5000] Train: 0.0439
Time:0.849705 Epoch [1278/5000] Train: 0.0439
Time:0.841917 Epoch [1279/5000] Train: 0.0439


KeyboardInterrupt: 